In [ ]:
# 초안 - 검색 + 본문 클롤링 구조만 버전

In [2]:
# 03_neil_keyword_pipeline.ipynb (예시 이름)

import pandas as pd
from collections import Counter
from kiwipiepy import Kiwi

kiwi = Kiwi()

In [3]:
# 1) 길이/넓이 관련
length_keywords = [
    "짧은손톱", "긴손톱", "넓은", "좁은", "가늘", "두껍",
    "세로", "가로", "비율"
]

# 2) 손톱 형태 관련
shape_keywords = [
    "라운드", "오벌", "스퀘어", "스퀘어오벌", "아몬드"
]

# 3) 네일 디자인 (한글 + 영어)
design_keywords = [
    "아트", "art",
    "풀코트", "풀 코트", "fullcoat", "full coat",
    "프렌치", "french",
    "쓰리디", "3d", "3D"
]

# 4) 색상 관련
color_keywords = [
    "시럽", "프렌치", "메탈", "글리터", "누드톤",
    "화이트", "핑크", "버건디", "골드", "베이지"
]

keyword_dict = {
    "length": length_keywords,
    "shape": shape_keywords,
    "design": design_keywords,
    "color": color_keywords,
}

In [4]:
# 분석 1
def tokenize(text: str):
    """본문 문자열 → 명사 계열 토큰 리스트"""
    if not isinstance(text, str):
        return []
    tokens = kiwi.tokenize(text)
    # 명사류(NN*) + 길이 2자 이상만 사용
    return [t.form for t in tokens if len(t.form) > 1 and t.tag.startswith("N")]

In [5]:
#분석 2
def count_keywords(df: pd.DataFrame, text_col: str, keyword_dict: dict):
    """
    df[text_col] 기준으로 카테고리별 키워드 빈도 계산
    """
    cat_counters = {cat: Counter() for cat in keyword_dict.keys()}

    for _, row in df.iterrows():
        text = row.get(text_col, "")
        tokens = tokenize(text)

        for cat, kw_list in keyword_dict.items():
            for tok in tokens:
                if any(k in tok for k in kw_list):
                    cat_counters[cat][tok] += 1

    return cat_counters


In [6]:
#분석 3
def counters_to_df(cat_counters: dict):
    """
    {"length": Counter(...), ...} → 카테고리별 DataFrame dict
    """
    result = {}
    for cat, counter in cat_counters.items():
        items = [{"keyword": k, "count": v} for k, v in counter.most_common()]
        result[cat] = pd.DataFrame(items)
    return result


In [ ]:
# (01 파일 안에 있다고 가정)
# def run_naver_blog_search(query_list: list) -> pd.DataFrame:
#     ...
#     return df_search


In [ ]:
# from naver_search_01 import run_naver_blog_search

def step1_search(query_list):
    """
    01 파일의 검색 코드 재사용.
    반환 df 예시 컬럼:
    - search_keyword
    - title
    - snippet
    - blog_url  (← 이게 중요)
    """
    df_search = run_naver_blog_search(query_list)
    return df_search

In [ ]:
# (02 파일 안에 있다고 가정)
# def crawl_blog_contents(df_links: pd.DataFrame) -> pd.DataFrame:
#     ...
#     return df_contents


In [ ]:
# from blog_contents_02 import crawl_blog_contents

def step2_crawl(df_search):
    """
    01 단계 결과(df_search)를 기반으로
    02 파일의 본문 크롤러를 호출.
    
    기대하는 df_search 컬럼:
    - blog_url
    - search_keyword (있으면 좋음)
    """
    df_contents = crawl_blog_contents(df_search)
    return df_contents


In [ ]:
def analyze_from_search(df_search):
    """
    01 검색 결과 df 기반 snippet/요약 텍스트 분석
    """
    cat_counters = count_keywords(df_search, text_col="snippet", keyword_dict=keyword_dict)
    cat_dfs = counters_to_df(cat_counters)
    return cat_dfs


In [ ]:
def analyze_from_contents(df_contents):
    """
    02 본문 크롤링 결과 df 기반 body 텍스트 분석
    """
    cat_counters = count_keywords(df_contents, text_col="body", keyword_dict=keyword_dict)
    cat_dfs = counters_to_df(cat_counters)
    return cat_dfs


In [ ]:
# 1. 검색어 리스트 정의
query_list = ["자석 네일", "가을 네일", "시럽 네일"]

# 2. 01 검색 실행
df_search = step1_search(query_list)
print(df_search.head())

# 3. 02 본문 크롤링 실행 (링크 기준)
df_contents = step2_crawl(df_search)
print(df_contents.head())

# 4-1. 검색 결과 기반 키워드 분석
search_cat_dfs = analyze_from_search(df_search)
# 예: search_cat_dfs["length"].head()

# 4-2. 본문 기반 키워드 분석
contents_cat_dfs = analyze_from_contents(df_contents)
# 예: contents_cat_dfs["color"].head()